In [2]:
import sqlite3
import math
import folium
import pandas as pd
import geopandas
import plotly.express as px

In [3]:
# Extract data for overall average rental prices, creating 'heatmap' style groupings and draw results on generated map
center = [1.3380, 103.8200]
my_map = folium.Map(
    tiles = 'Cartodb Positron',
    location = center,
    zoom_start = 12,
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False
    )

gdf = geopandas.read_file('townsg.json')
rent_group = []
col_group = []
for v in gdf['avg_rent']:
    rent_group.append(math.ceil(v/100))
    if v <= 2400.00: col_group.append(1)
    if v > 2400.00 and v <= 2600.00: col_group.append(2)
    if v > 2600.00 and v <= 2800.00: col_group.append(3)
    if v > 2800.00: col_group.append(4)

gdf['rent_group'] = rent_group
gdf['col_group'] = col_group

gdf['avg_rent'] = gdf['avg_rent'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_1room'] = gdf['avg_rent_1room'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_2room'] = gdf['avg_rent_2room'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_3room'] = gdf['avg_rent_3room'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_4room'] = gdf['avg_rent_4room'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_5room'] = gdf['avg_rent_5room'].apply(lambda x: '${:,.2f}'.format(x))
gdf['avg_rent_executive'] = gdf['avg_rent_executive'].apply(lambda x: '${:,.2f}'.format(x))

colors = ['black', 'green', 'yellow', 'orange', 'red']

folium.GeoJson(
    gdf,
    name='Avg. HDB Rental Cost',
    marker=folium.Circle(radius=900, fill_color='green', color='black', weight=0.5, fill_opacity=0.3),
    tooltip=folium.GeoJsonTooltip( 
        fields=[
            'name',
            'avg_rent',
            'avg_rent_1room',
            'avg_rent_2room',
            'avg_rent_3room',
            'avg_rent_4room',
            'avg_rent_5room',
            'avg_rent_executive'
            ],
        aliases=[
            'Town:',
            'Avg. Monthly Rent (Overall):',
            'Avg. Monthly Rent (1-Room):',
            'Avg. Monthly Rent (2-Room):',
            'Avg. Monthly Rent (3-Room):',
            'Avg. Monthly Rent (4-Room):',
            'Avg. Monthly Rent (5-Room):',
            'Avg. Monthly Rent (Executive):'
            ]
        ),
    style_function=lambda x: {
        'fillColor': colors[x['properties']['col_group']],
        'radius': (x['properties']['rent_group'])*30,
        },
    highlight_function=lambda x: {'fillOpacity': 0.8}
).add_to(my_map)

my_map

From the map above, we can see that the overall average monthly rent for HDB apartments are expectedly highest in towns closest to Singapore's CBD area or in known affluent neighbourhoods such as Bukit Timah, Bukit Merah and Queenstown. However, we do also see higher rentals outside of the central zone such as Bishan and Pasir Ris. Even in areas furthest from the center such as Jurong West, Choa Chu Kang and Punggol are seeing competing rental prices to the most expensive neighbourhoods.

In [4]:
# Extract data from SQLite to calculate average monthly rental units and prices
conn = sqlite3.connect('rental.sqlite')
df = pd.read_sql_query('''
                  SELECT
                    flat_type,
                    month,
                    rental_units,
                    ROUND(((rental_units/prev_rental_units)-1) ,2) AS rental_units_change,
                    ROUND(monthly_rent,2) AS monthly_rent,
                    ROUND(((monthly_rent/prev_monthly_rent)-1) ,2) AS monthly_rent_change
                  FROM (
                    SELECT *,
                      CAST(LAG(rental_units) OVER (PARTITION BY flat_type ORDER BY month) AS FLOAT) AS prev_rental_units,
                      CAST(LAG(monthly_rent) OVER (PARTITION BY flat_type ORDER BY month) AS FLOAT) AS prev_monthly_rent
                    FROM (
                      SELECT
                        flat_type,
                        DATE((rent_approval_date || '-01')) AS month,
                        COUNT(*) AS rental_units,
                        AVG(monthly_rent) AS monthly_rent
                      FROM rental
                      GROUP BY 1,2
                    )
                  )
                  ORDER BY 1,2
                  ''', conn)

In [5]:
# Chart rental units available per month
fig_rental_units = px.line(df, x='month', y='rental_units', color='flat_type',
                           title='HDB Rental units per month',
                           labels={'month': 'Month', 'rental_units': '# of Rental Units', 'flat_type': 'Flat Type'},
                           hover_data={'month':False})
fig_rental_units.update_layout(yaxis_tickformat=',', hovermode='x unified')
fig_rental_units.update_traces(hovertemplate='%{y} units')
fig_rental_units.show()

# Chart change rate for rental units available
fig_rental_units_change = px.bar(df, x='month', y='rental_units_change', color='flat_type',
                                 title='Change in monthly HDB rental units',
                                 labels={'month': 'Month', 'rental_units_change': 'Rental Units Change Rate', 'flat_type': 'Flat Type'},
                                 hover_data={'month':False})
fig_rental_units_change.update_layout(yaxis_tickformat='.1%', hovermode='x unified')
fig_rental_units_change.update_traces(hovertemplate='%{y} units')
fig_rental_units_change.show()

With the exception of 1-Room apartments which are rarer in Singapore, we do see the availability or demand for HDB rentals generally stagnating - averaging at about **544** units rented out each month. This could indicate that there is a lack of increasing landlords (i.e. locals unwilling to invest in HDB for rental purposes) or that demand has not changed since 2021.

In [6]:
# Chart average monthly rent per month
fig_avg_rent = px.line(df, x='month', y='monthly_rent', color='flat_type',
                       title='Avg. monthly rent for HDB units by Type',
                       labels={'month': 'Month', 'monthly_rent': 'Avg. Monthly Rent', 'flat_type': 'Flat Type'},
                       hover_data={'month':False})
fig_avg_rent.update_layout(yaxis_tickformat='$,.2f', hovermode='x unified')
fig_avg_rent.update_traces(hovertemplate='%{y}')
fig_avg_rent.show()

# Chart change rate for average monthly rent change rate
fig_avg_rent_change = px.bar(df, x='month', y='monthly_rent_change', color='flat_type',
                             title='Change in avg. monthly rent for HDB units',
                             labels={'month': 'Month', 'monthly_rent_change': 'Avg. Monthly Rent Change Rate', 'flat_type': 'Flat Type'},
                             hover_data={'month':False})
fig_avg_rent_change.update_layout(yaxis_tickformat='.1%', hovermode='x unified')
fig_avg_rent_change.update_traces(hovertemplate='%{y}')
fig_avg_rent_change.show()

Despite a stagnating availability or demand in HDB rental units, prices have soared with most unit types except 1-Room apartments seeing more increases in rental prices month-on-month than a decrease in price. Especially for 3-Room and 4-Room units which saw more increments in monthly rent since 2021.

Across the board, all the unit types saw the highest rental price hikes during the mid to late half of 2022 - admidst the Covid pandemic - inverse to the change rate in HDB rental units. This is mostly likely due to the drop in demand or availability during the pandemic leading to most foreign workers and expats (i.e. people most likely to rent units) leaving the country during this period.

In [7]:
# Extract data for average monthly rent prices by flat-type for earliest and latest year comparison
conn = sqlite3.connect('rental.sqlite')
df = pd.read_sql_query('''
						SELECT *,
							CASE
								WHEN avg_rent_change IS NULL THEN ('$' || ROUND(avg_rent,0) || '<br>( - )')
								ELSE ('$' || ROUND(avg_rent,0) || '<br>(' || avg_rent_change || '%)') 
							END AS avg_rent_text
						FROM (
							SELECT *,
								ROUND(((avg_rent/prev_avg_rent)-1)*100,1) AS avg_rent_change
							FROM (
								SELECT *,
									CAST(LAG(avg_rent) OVER (PARTITION BY flat_type ORDER BY year) AS FLOAT) AS prev_avg_rent
								FROM (
									SELECT
										flat_type,
										year,
										ROUND(AVG(monthly_rent),2) AS avg_rent
									FROM (
										SELECT
											flat_type,
											STRFTIME('%Y', DATE((rent_approval_date || '-01'))) AS year,
											DATE((rent_approval_date || '-01')) AS month,
											AVG(monthly_rent) AS monthly_rent
										FROM rental
										GROUP BY 1,2,3
									)
									WHERE year IN ('2021','2024')
									GROUP BY 1,2
								)
							)
						)
                  		''', conn)

fig_yearly_rent = px.bar(df, x='year', y='avg_rent', color='flat_type', barmode='group', text='avg_rent_text',
                         title='Avg. monthly rent per year',
                         labels={'year': 'Year', 'avg_rent': 'Avg. Monthly Rent', 'flat_type': 'Flat Type'})
fig_yearly_rent.update_layout(yaxis_tickformat='$,.2f', hovermode=False)
fig_yearly_rent.show()

Comparing the dataset from the earliest records in 2021 and the latest records in 2024, we can see how monthly rental prices have spiked, with almost all unit types (except for 1-Room units) seeing a 40%~50% increase in prices.